# Alpha

This notebook investigates the alpha of S&P stocks

### Import & Format Data

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
from datetime import datetime, timedelta

In [4]:
# S&P index
spy = pd.read_csv('../data/price/SPY.csv', index_col='Date', parse_dates=True)
print('SPY\t\t', spy.shape)

# 10y Treasury yeild
t_bill = pd.read_csv('../data/price/%5ETNX.csv', index_col='Date', parse_dates=True)
t_bill = t_bill.loc[spy.index.intersection(t_bill.index)] # Only select dates valid from SPY
print('10y TBill (raw)\t', t_bill.shape)

diff_index = spy.index.difference(t_bill.index)
nan_df = pd.DataFrame([np.full(6, np.nan) for x in range(diff_index.size)], index=diff_index, columns=t_bill.columns)
t_bill = t_bill.append(nan_df).sort_index()
t_bill = t_bill.interpolate(method='time')

assert t_bill[t_bill.Close.isnull()].index.size == 0 # check all nan's have been filled
print('10y TBill\t', t_bill.shape)

SPY		 (6690, 6)
10y TBill (raw)	 (6671, 6)
10y TBill	 (6690, 6)


In [5]:
# get tickers
tickers = pd.read_csv('../data/spy/tickers.csv', header=None)[1]

# Load all stocks
all_stocks = {}
for ticker in tickers:
    stock = pd.read_csv('../data/price/{}.csv'.format(ticker), index_col='Date', parse_dates=True)
    all_stocks[ticker] = stock

print(len(all_stocks))

505


In [6]:
df = pd.DataFrame(spy['Adj Close'].rename('price'))
df['pct_change'] = df.price.pct_change()
df['log_return'] = np.log(df.price).diff()
df = df.iloc[1:]
y = df['log_return']

In [7]:
from ..data.info import info

ValueError: attempted relative import beyond top-level package